In [1]:
import cv2
import os

video_dirs = [
    r"f:\SKRIPSI\TA\DROZY\videos_i8"]

# Target FPS dan minimum frame
target_fps = 15
min_frames = 8000

selected_videos = []

# Iterasi melalui semua direktori
for video_dir in video_dirs:
    # Iterasi melalui semua file di direktori video
    for filename in os.listdir(video_dir):
        if filename.lower().endswith((".mov", ".mp4")):  # This handles both .MOV and .mov
            video_path = os.path.join(video_dir, filename)
            cap = cv2.VideoCapture(video_path)
            
            # Mendapatkan frame rate dan jumlah total frame
            fps = cap.get(cv2.CAP_PROP_FPS)
            frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            
            # Memeriksa apakah video memenuhi kriteria
            if fps >= target_fps and frame_count > min_frames:
                selected_videos.append((video_dir, filename))
            
            cap.release()

# Mencetak video yang memenuhi kriteria
print("Video yang memenuhi kriteria:")
for video_dir, video in selected_videos:
    print(f"{video} in {video_dir}")
    
print("Total video yang memenuhi kriteria:", len(selected_videos))


Video yang memenuhi kriteria:
1-1.mp4 in f:\SKRIPSI\TA\DROZY\videos_i8
1-2.mp4 in f:\SKRIPSI\TA\DROZY\videos_i8
1-3.mp4 in f:\SKRIPSI\TA\DROZY\videos_i8
10-1.mp4 in f:\SKRIPSI\TA\DROZY\videos_i8
10-3.mp4 in f:\SKRIPSI\TA\DROZY\videos_i8
11-1.mp4 in f:\SKRIPSI\TA\DROZY\videos_i8
11-2.mp4 in f:\SKRIPSI\TA\DROZY\videos_i8
11-3.mp4 in f:\SKRIPSI\TA\DROZY\videos_i8
12-1.mp4 in f:\SKRIPSI\TA\DROZY\videos_i8
13-1.mp4 in f:\SKRIPSI\TA\DROZY\videos_i8
13-2.mp4 in f:\SKRIPSI\TA\DROZY\videos_i8
14-1.mp4 in f:\SKRIPSI\TA\DROZY\videos_i8
14-2.mp4 in f:\SKRIPSI\TA\DROZY\videos_i8
14-3.mp4 in f:\SKRIPSI\TA\DROZY\videos_i8
2-1.mp4 in f:\SKRIPSI\TA\DROZY\videos_i8
2-2.mp4 in f:\SKRIPSI\TA\DROZY\videos_i8
2-3.mp4 in f:\SKRIPSI\TA\DROZY\videos_i8
3-1.mp4 in f:\SKRIPSI\TA\DROZY\videos_i8
3-2.mp4 in f:\SKRIPSI\TA\DROZY\videos_i8
3-3.mp4 in f:\SKRIPSI\TA\DROZY\videos_i8
4-1.mp4 in f:\SKRIPSI\TA\DROZY\videos_i8
4-2.mp4 in f:\SKRIPSI\TA\DROZY\videos_i8
4-3.mp4 in f:\SKRIPSI\TA\DROZY\videos_i8
5-1.mp4 in f:\SK

In [2]:
import cv2
import dlib
import numpy as np
from imutils import face_utils
from scipy.spatial import distance as dist
import csv

# Initialize dlib's face detector and load the facial landmark predictor
p = "F:\SKRIPSI\TA\Kartivel\Driver-Drowsiness-Detection-main\shape_predictor_68_face_landmarks.dat"  # Provide the correct path
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(p)

# Function to calculate Eye Aspect Ratio (EAR)
def eye_aspect_ratio(eye):
    # vertical eye landmarks
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    # horizontal eye landmark
    C = dist.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

# Function to calculate Mouth Aspect Ratio (MAR)
def mouth_aspect_ratio(mouth):
    # vertical mouth landmarks
    A = dist.euclidean(mouth[2], mouth[6])  # 51, 59
    B = dist.euclidean(mouth[4], mouth[5])  # 53, 57
    # horizontal mouth landmarks
    C = dist.euclidean(mouth[0], mouth[6])  # 49, 55
    mar = (A + B) / (2.0 * C)
    return mar


# Initialize variables
EYE_AR_THRESH = 0.3
TOTAL_BLINKS = 0
TOTAL_FRAMES = 0
CLOSED_EYES_FRAME_COUNT = 1
frame_data = []

# Segment parameters
window_size = 300  # Number of frames in a segment
step_size = 150  # Number of frames to step for the next segment
segments_ear = []
segments_mar = []
segments_perclos = []

def mean_of_segment(segment):
    return sum(segment) / len(segment)

video_path = r"F:\SKRIPSI\TA\DROZY\videos_i8\3-3.mp4"  # Use the correct path or 0 for webcam
cap = cv2.VideoCapture(video_path)

# Get the base name of the video file without the extension
video_basename = os.path.basename(video_path)
video_title, _ = os.path.splitext(video_basename)

# Define the CSV file name based on the video title
csv_filename = f"{video_title}_data_analysis.csv"


while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to grayscale and detect faces
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray, 0)

    for face in faces:
        shape = predictor(gray, face)
        shape = face_utils.shape_to_np(shape)

        # Extract the left and right eye coordinates, then calculate EAR for both
        leftEye = shape[42:48]
        rightEye = shape[36:42]
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
        
        ear = (leftEAR + rightEAR) / 2.0

        # Extract the mouth coordinates, then calculate MAR
        mouth = shape[60:68]  # The indices might vary depending on the facial landmark map
        mar = mouth_aspect_ratio(mouth)

        # Check if EAR is below the blink threshold
        if ear < EYE_AR_THRESH:
            CLOSED_EYES_FRAME_COUNT += 1
        else:
            if CLOSED_EYES_FRAME_COUNT >= 3:  # Typically, a blink is 0.1-0.4 sec
                TOTAL_BLINKS += 1
            CLOSED_EYES_FRAME_COUNT = 0
            
        # Draw contours on the eyes and mouth
        cv2.drawContours(frame, [cv2.convexHull(leftEye)], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [cv2.convexHull(rightEye)], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [cv2.convexHull(mouth)], -1, (0, 255, 0), 1)

        # Calculate PERCLOS
        # Before calculating PERCLOS, check if TOTAL_FRAMES is greater than 0
        if TOTAL_FRAMES > 0:
            PERCLOS = CLOSED_EYES_FRAME_COUNT / float(TOTAL_FRAMES)
        else:
            PERCLOS = 0 # Set PERCLOS to 0 if no frames have been processed


        # Display EAR, MAR, and PERCLOS on the frame
        cv2.putText(frame, f"EAR: {ear:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, f"MAR: {mar:.2f}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, f"PERCLOS: {PERCLOS:.2f}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    
    frame_data.append((TOTAL_FRAMES, ear, mar, PERCLOS))

    # Show the frame
    cv2.imshow("Frame", frame)

    # Increment the total number of frames processed
    TOTAL_FRAMES += 1

    # Exit loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Clean up
cap.release()
cv2.destroyAllWindows()

with open(csv_filename, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Frame", "EAR", "MAR", "PERCLOS"])  # Write the header
    writer.writerows(frame_data)  # Write the frame data

    print(f"Data written to {csv_filename}")

Data written to 3-3_data_analysis.csv


In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import csv

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=False,
    max_num_faces=1,
    min_detection_confidence=0.5
)

def eye_openness(eye_region):
    gray_eye = cv2.cvtColor(eye_region, cv2.COLOR_BGR2GRAY)
    _, binary_eye = cv2.threshold(gray_eye, 50, 255, cv2.THRESH_BINARY_INV)
    white_pixels = cv2.countNonZero(binary_eye)

   #Normalisasi dengan membagi total pixel dibagian mata
    openness = white_pixels / (binary_eye.shape[0] * binary_eye.shape[1])
    return openness, binary_eye

# ekstrak fitur mata 
def get_eye_region(image, landmarks, eye_indices):
    #mengambil titik koordinat landmark
    eye_points = np.array([(int(landmarks.landmark[i].x * image.shape[1]), int(landmarks.landmark[i].y * image.shape[0])) for i in eye_indices])
    eye_points = eye_points.reshape(-1, 1, 2)
    # menghitung bonding box
    x, y, w, h = cv2.boundingRect(eye_points)
    eye_region = image[y:y+h, x:x+w]
    return eye_region

def mouth_aspect_ratio(landmarks):
    # Indices for landmarks of the outer corners of the mouth
    left_mouth_corner = np.array([landmarks[61].x, landmarks[61].y])  # Left corner
    right_mouth_corner = np.array([landmarks[291].x, landmarks[291].y])  # Right corner

    # Indices for landmarks of the upper and lower inner lip
    upper_inner_lip = np.array([landmarks[13].x, landmarks[13].y])  # Upper inner lip
    lower_inner_lip = np.array([landmarks[14].x, landmarks[14].y])  # Lower inner lip

    # Calculate the distances
    horiz_dist = np.linalg.norm(left_mouth_corner - right_mouth_corner)
    vert_dist = np.linalg.norm(upper_inner_lip - lower_inner_lip)

    # Calculate MAR
    mar = vert_dist / horiz_dist * 1.5
    return mar


#masih percobaan
# def draw_facial_features(image, landmarks, eye_indices, mouth_indices):
# #     # Get the points for the eyes
#      left_eye = np.array([(landmarks[i].x * image.shape[1], landmarks[i].y * image.shape[0]) for i in eye_indices[0]], dtype=np.int32)
#      right_eye = np.array([(landmarks[i].x * image.shape[1], landmarks[i].y * image.shape[0]) for i in eye_indices[1]], dtype=np.int32)

#      # Draw the polylines for the eyes
#      cv2.polylines(image, [left_eye], True, (0, 255, 0), 2)
#      cv2.polylines(image, [right_eye], True, (0, 255, 0), 2)

#      # Get the points for the mouth
#      mouth_outer = np.array([(landmarks[i].x * image.shape[1], landmarks[i].y * image.shape[0]) for i in mouth_indices], dtype=np.int32)

# #     # Draw the lines for the mouth
#      cv2.polylines(image, [mouth_outer], True, (0, 255, 0), 2)

# #


# indeks mata di mp 468 titik
LEFT_EYE_INDICES = [33, 133, 160, 159, 158, 144, 145, 153, 154, 155]
RIGHT_EYE_INDICES = [362, 263, 387, 386, 385, 373, 374, 380, 381, 382]
MOUTH_INDICES = [61, 146, 91, 181, 84, 17, 314, 405, 321, 375, 291, 308, 402, 318, 324, 308]


video_path = r"F:\SKRIPSI\TA\DROZY\FixedDataset\1-1.mp4"  # Use the correct path or 0 for webcam
cap = cv2.VideoCapture(video_path)

# Get the base name of the video file without the extension
video_basename = os.path.basename(video_path)
video_title, _ = os.path.splitext(video_basename)

# Define the CSV file name based on the video title
csv_filename = f"{video_title}_data_analysis.csv"

fps = cap.get(cv2.CAP_PROP_FPS)
delay_between_frames = int(1000 / fps)
data_list = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Reached the end of the video or the video cannot be read.")
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(frame_rgb)

    # Frame number
    frame_number = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            landmarks = face_landmarks.landmark
            # mengambil bagian mata
            left_eye_region = get_eye_region(frame, face_landmarks, LEFT_EYE_INDICES)
            right_eye_region = get_eye_region(frame, face_landmarks, RIGHT_EYE_INDICES)
            #draw_facial_features(frame, face_landmarks.landmark, (LEFT_EYE_INDICES, RIGHT_EYE_INDICES), MOUTH_INDICES)

            # Calculate eye openness
            left_openness, left_binary_eye = eye_openness(left_eye_region)
            right_openness, right_binary_eye = eye_openness(right_eye_region)
            openness = (left_openness + right_openness)/2
 
            mar = mouth_aspect_ratio(landmarks)

            data_list.append([cap.get(cv2.CAP_PROP_POS_FRAMES), openness, mar])   
            # Display the openness and the binary eye image
            cv2.imshow('Left Eye Binary', left_binary_eye)
            cv2.imshow('Right Eye Binary', right_binary_eye)
            #cv2.putText(image, f'left Eye Openness: {left_openness:.2f}', (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
           # cv2.putText(image, f'Right Eye Openness: {right_openness:.2f}', (20, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(frame, f'Eye Openness: {openness:.2f}', (10, 35), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0),1)
            cv2.putText(frame, f'MAR: {mar:.2f}', (10, 65), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 1)
    else:
        # No landmarks detected, so set values to None or np.nan
        left_openness, right_openness, mar = None, None, None 
    # Show the image
    cv2.imshow('MediaPipe Face Mesh', frame)
    if cv2.waitKey(delay_between_frames) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()
with open(csv_filename, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    # Write the header
    csvwriter.writerow(['Frame', 'Openness', 'MAR'])
    # Write the data
    csvwriter.writerows(data_list)
    print(f"Data saved to {csv_filename}")

print(f"Processed {frame_number} frames from {video_title}")